In [4]:
import pickle
import tkinter as tk
from tkinter import filedialog
import pickle
import shap
import pandas as pd

**Open a PKL file and get its contents**

In [5]:
def load_pickle_file():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Pickle files", "*.pkl")])
    
    if file_path:
        with open(file_path, 'rb') as file:
            data = pickle.load(file)
        root.destroy()  # Close the tkinter root window
        return data
    else:
        print("No file selected.")
        root.destroy()  # Close the tkinter root window
        return None

In [7]:
data = load_pickle_file()
data['model']

2024-02-01 11:55:54.969 python[73430:2791981] +[CATransaction synchronize] called within transaction
2024-02-01 11:55:55.009 python[73430:2791981] +[CATransaction synchronize] called within transaction
2024-02-01 11:55:56.988 python[73430:2791981] +[CATransaction synchronize] called within transaction


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

**Get the data from HN_P2C.csv into a dataframe**

In [11]:
df = pd.read_csv('./HN_P2C.csv')
df_new = df[df['CLabel'].isin(['Healthy', 'Cancer'])]
df_new.to_csv('df_new.csv', index=False)
df_new.head()
features_of_interest = [
    'lifet_avg_ch1', 'lifet_avg_ch2', 'lifet_avg_ch3', 
    'int_ratio_ch1', 'int_ratio_ch2', 'int_ratio_ch3',
    'Laguerre_coeffs_1_ch1', 'Laguerre_coeffs_2_ch1', 'Laguerre_coeffs_3_ch1', 
    'Laguerre_coeffs_4_ch1', 'Laguerre_coeffs_5_ch1', 'Laguerre_coeffs_6_ch1', 
    'Laguerre_coeffs_7_ch1', 'Laguerre_coeffs_8_ch1', 'Laguerre_coeffs_9_ch1', 
    'Laguerre_coeffs_10_ch1', 'Laguerre_coeffs_11_ch1', 'Laguerre_coeffs_12_ch1',
    'Laguerre_coeffs_1_ch2', 'Laguerre_coeffs_2_ch2', 'Laguerre_coeffs_3_ch2', 
    'Laguerre_coeffs_4_ch2', 'Laguerre_coeffs_5_ch2', 'Laguerre_coeffs_6_ch2', 
    'Laguerre_coeffs_7_ch2', 'Laguerre_coeffs_8_ch2', 'Laguerre_coeffs_9_ch2', 
    'Laguerre_coeffs_10_ch2', 'Laguerre_coeffs_11_ch2', 'Laguerre_coeffs_12_ch2',
    'Laguerre_coeffs_1_ch3', 'Laguerre_coeffs_2_ch3', 'Laguerre_coeffs_3_ch3', 
    'Laguerre_coeffs_4_ch3', 'Laguerre_coeffs_5_ch3', 'Laguerre_coeffs_6_ch3', 
    'Laguerre_coeffs_7_ch3', 'Laguerre_coeffs_8_ch3', 'Laguerre_coeffs_9_ch3', 
    'Laguerre_coeffs_10_ch3', 'Laguerre_coeffs_11_ch3', 'Laguerre_coeffs_12_ch3'
]
df = df_new[features_of_interest]
df.head()

Columns (12,17,20) have mixed types. Specify dtype option on import or set low_memory=False.


,lifet_avg_ch1,lifet_avg_ch2,lifet_avg_ch3,int_ratio_ch1,int_ratio_ch2,int_ratio_ch3,Laguerre_coeffs_1_ch1,Laguerre_coeffs_2_ch1,Laguerre_coeffs_3_ch1,Laguerre_coeffs_4_ch1,...,Laguerre_coeffs_3_ch3,Laguerre_coeffs_4_ch3,Laguerre_coeffs_5_ch3,Laguerre_coeffs_6_ch3,Laguerre_coeffs_7_ch3,Laguerre_coeffs_8_ch3,Laguerre_coeffs_9_ch3,Laguerre_coeffs_10_ch3,Laguerre_coeffs_11_ch3,Laguerre_coeffs_12_ch3
1369,3.677834,4.372698,4.673487,0.383847,0.401777,0.214376,2.287684,0.045487,0.345883,-0.038626,...,0.177757,-0.046781,0.050329,-0.042205,0.011809,-0.012513,0.004941,-0.002295,0.001451,-0.000368
1370,3.725055,4.311057,4.678097,0.399188,0.388146,0.212665,1.963396,0.122349,0.386040,0.060966,...,0.163542,-0.023193,0.078119,0.008415,0.037673,0.003595,0.011348,0.000742,0.001862,-0.000485
1371,3.809186,4.337621,4.262062,0.391038,0.394803,0.214159,1.712471,0.112410,0.298065,0.003290,...,0.158346,-0.023289,0.039994,-0.022972,0.003650,-0.004216,-0.001125,-0.000643,-0.000201,-0.000680
1372,3.822145,4.295335,3.905487,0.389809,0.391994,0.218196,1.871546,0.083860,0.299186,-0.000846,...,0.165047,-0.010450,0.045931,-0.018878,0.000380,-0.006706,-0.001397,-0.002114,0.000458,-0.000604
1373,3.719731,4.497713,4.277077,0.370316,0.402411,0.227272,2.116811,0.068791,0.330150,-0.037806,...,0.204938,-0.047447,0.051589,-0.031742,0.002631,-0.009467,-0.000464,-0.000858,0.000830,0.000674


**Call Function for the user to select a .pkl file**

In [12]:
data = load_pickle_file()

if data is not None:
    # model_features = [
    # 'lifet_avg_ch1', 'lifet_avg_ch2', 'lifet_avg_ch3', 
    # 'spec_int_ch1', 'spec_int_ch2', 'spec_int_ch3', 
    # 'int_ratio_ch1', 'int_ratio_ch2', 'int_ratio_ch3', 
    # 'snr_ch1', 'snr_ch2', 'snr_ch3', 
    # 'Laguerre_coeffs_1_ch1', 'Laguerre_coeffs_2_ch1', 'Laguerre_coeffs_3_ch1', 
    # 'Laguerre_coeffs_4_ch1', 'Laguerre_coeffs_5_ch1', 'Laguerre_coeffs_6_ch1', 
    # 'Laguerre_coeffs_7_ch1', 'Laguerre_coeffs_8_ch1', 'Laguerre_coeffs_9_ch1', 
    # 'Laguerre_coeffs_10_ch1', 'Laguerre_coeffs_11_ch1', 'Laguerre_coeffs_12_ch1', 
    # 'Laguerre_coeffs_1_ch2', 'Laguerre_coeffs_2_ch2', 'Laguerre_coeffs_3_ch2', 
    # 'Laguerre_coeffs_4_ch2', 'Laguerre_coeffs_5_ch2', 'Laguerre_coeffs_6_ch2', 
    # 'Laguerre_coeffs_7_ch2', 'Laguerre_coeffs_8_ch2', 'Laguerre_coeffs_9_ch2', 
    # 'Laguerre_coeffs_10_ch2', 'Laguerre_coeffs_11_ch2', 'Laguerre_coeffs_12_ch2', 
    # 'Laguerre_coeffs_1_ch3', 'Laguerre_coeffs_2_ch3', 'Laguerre_coeffs_3_ch3', 
    # 'Laguerre_coeffs_4_ch3', 'Laguerre_coeffs_5_ch3', 'Laguerre_coeffs_6_ch3', 
    # 'Laguerre_coeffs_7_ch3', 'Laguerre_coeffs_8_ch3', 'Laguerre_coeffs_9_ch3', 
    # 'Laguerre_coeffs_10_ch3', 'Laguerre_coeffs_11_ch3', 'Laguerre_coeffs_12_ch3'
    # ]
    model = data['model']
    dataset = data['post_processing_data']
    # adjusted_dataset = dataset.drop(['WLI_x_coord', 'score', 'WLI_y_coord', 'gt', 'WLI_frame_n', 'Case', 'Run'], axis=1, errors='ignore')
    # adjusted_dataset = adjusted_dataset[model_features]
    

2024-01-29 08:22:28.624 python[9260:259042] +[CATransaction synchronize] called within transaction


In [13]:
model
dataset

,Case,Run,snr_ch1,snr_ch2,snr_ch3,WLI_frame_n,WLI_x_coord,WLI_y_coord,score,gt
511612,170,2,55.704300,55.393034,54.403223,15,631,99,0.002971,0
511613,170,2,53.731903,54.760765,51.727847,15,631,99,0.000308,0
511614,170,2,54.003230,55.346427,53.164215,16,639,102,0.002821,0
511615,170,2,51.710158,51.210164,50.018434,16,639,102,0.003646,0
511616,170,2,50.549549,51.228330,53.102477,16,639,102,0.010020,0
...,...,...,...,...,...,...,...,...,...,...
512782,170,2,53.557013,52.332017,54.126046,1711,513,185,0.128708,0
512783,170,2,52.147328,53.499322,50.886574,1711,513,185,0.112700,0
512784,170,2,53.577052,54.818158,55.389933,1711,513,185,0.210239,0
512785,170,2,52.273561,53.361657,51.863266,1711,513,185,0.225831,0


In [16]:
if data is not None:
    # cols_to_drop = ['WLI_x_coord', 'score', 'WLI_y_coord', 'gt', 'WLI_frame_n', 'Case', 'Run']
    # new_data = dataset.drop(columns=cols_to_drop, errors='ignore')
    try:
        explainer = shap.TreeExplainer(model)
        shap_values = explainer.shap_values(df)
    
        # Visualize the SHAP values
        shap.summary_plot(shap_values, df)
    except Exception as e:
        print("Error during SHAP analysis:", e)

[08:39:48] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.


Error during SHAP analysis: This reshape error is often caused by passing a bad data matrix to SHAP. See https://github.com/shap/shap/issues/580.


**Code to Generate the Error**

In [20]:
try:
    # Adjust the slicing as per your dataset
    sample_data = dataset.iloc[0:10, :]
    predictions = model.predict(sample_data)
    print("Sample Predictions:", predictions)
except Exception as e:
    print("Error during model prediction:", e)

Error during model prediction: feature_names mismatch: ['lifet_avg_ch1', 'lifet_avg_ch2', 'lifet_avg_ch3', 'spec_int_ch1', 'spec_int_ch2', 'spec_int_ch3', 'int_ratio_ch1', 'int_ratio_ch2', 'int_ratio_ch3', 'snr_ch1', 'snr_ch2', 'snr_ch3', 'Laguerre_coeffs_1_ch1', 'Laguerre_coeffs_2_ch1', 'Laguerre_coeffs_3_ch1', 'Laguerre_coeffs_4_ch1', 'Laguerre_coeffs_5_ch1', 'Laguerre_coeffs_6_ch1', 'Laguerre_coeffs_7_ch1', 'Laguerre_coeffs_8_ch1', 'Laguerre_coeffs_9_ch1', 'Laguerre_coeffs_10_ch1', 'Laguerre_coeffs_11_ch1', 'Laguerre_coeffs_12_ch1', 'Laguerre_coeffs_1_ch2', 'Laguerre_coeffs_2_ch2', 'Laguerre_coeffs_3_ch2', 'Laguerre_coeffs_4_ch2', 'Laguerre_coeffs_5_ch2', 'Laguerre_coeffs_6_ch2', 'Laguerre_coeffs_7_ch2', 'Laguerre_coeffs_8_ch2', 'Laguerre_coeffs_9_ch2', 'Laguerre_coeffs_10_ch2', 'Laguerre_coeffs_11_ch2', 'Laguerre_coeffs_12_ch2', 'Laguerre_coeffs_1_ch3', 'Laguerre_coeffs_2_ch3', 'Laguerre_coeffs_3_ch3', 'Laguerre_coeffs_4_ch3', 'Laguerre_coeffs_5_ch3', 'Laguerre_coeffs_6_ch3', 'Lagu